In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [5]:
DROP TABLE IF EXISTS Antivax_Trends;

CREATE TABLE Antivax_Trends (
    Periodo INTEGER,
    antivax INTEGER,
    vacina_causa_autismo INTEGER,
    anti_vacina INTEGER,
    anti_vacinacao INTEGER,
    Soma INTEGER,
    Popularidade INTEGER,
    PRIMARY KEY(Periodo)
) AS SELECT
    Periodo,
    antivax,
    vacina-causa-autismo,
    anti-vacina,
    anti-vacinacao,
    Soma,
    Popularidade
FROM CSVREAD('../data/vacinas/trends-antivax.csv');

CREATE TABLE Immunization_Estimates (
    Id INTEGER NOT NULL,
    unicef_region VARCHAR,
    iso3 VARCHAR,
    country VARCHAR,
    vaccine VARCHAR,
    Ano INTEGER,
    Taxa INTEGER,
    PRMARY KEY(Id)
)AS SELECT
    Id,
    Unicef_region,
    Iso3,
    Country,
    Vaccine,
    Ano,
    Taxa
FROM CSVREAD('../data/vacinas/Immunization-estimates-2018-Brasil.csv')


org.h2.jdbc.JdbcSQLException:  Column "PERIODO" not found; SQL statement

In [3]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [6]:
SELECT Med.NomeMedicamento, COUNT(*) 
FROM Medicamento Med, UsoMedicamento Uso, Pessoa 
WHERE Med.Codigo = Uso.CodMedicamento AND Pessoa.Id = Uso.IdPessoa 
GROUP BY Med.NomeMedicamento

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [8]:
CREATE VIEW IF NOT EXISTS EXEMPLO AS
SELECT Med.Codigo, COUNT(*) AS qtdTratados
FROM Medicamento Med, UsoMedicamento Uso, Pessoa
WHERE Med.Codigo = Uso.CodMedicamento AND Pessoa.Id = Uso.IdPessoa
GROUP BY Med.Codigo;

SELECT Exemplo.Codigo
FROM EXEMPLO
WHERE Exemplo.QtdTratados = (SELECT MAX(Exemplo.QtdTratados) FROM EXEMPLO);

d00732

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [16]:
SELECT Med.Codigo
FROM Medicamento Med, UsoMedicamento Uso, Pessoa
WHERE Med.Codigo = Uso.CodMedicamento AND Pessoa.Id = Uso.IdPessoa 
GROUP BY Med.Codigo
HAVING COUNT(*) = SELECT MAX(qtdTratados) FROM (SELECT Med.Codigo, COUNT(*) AS qtdTratados
FROM Medicamento Med, UsoMedicamento Uso, Pessoa
WHERE Med.Codigo = Uso.CodMedicamento AND Pessoa.Id = Uso.IdPessoa 
GROUP BY Med.Codigo)

d00732

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [38]:
CREATE VIEW IF NOT EXISTS EXEMPLO2 AS (
SELECT Med.NomeMedicamento, COUNT(*) AS qtdTratados
FROM Medicamento Med, UsoMedicamento Uso, Pessoa
WHERE Med.Codigo = Uso.CodMedicamento AND Pessoa.Id = Uso.IdPessoa
GROUP BY Med.NomeMedicamento);


SELECT Exemplo2.NomeMedicamento
FROM EXEMPLO2
WHERE Exemplo2.QtdTratados = (SELECT MAX(Exemplo2.QtdTratados) FROM EXEMPLO2);

LISINOPRIL

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [25]:
SELECT AVG(NumMedicado)
FROM (SELECT Pessoa.Id, COUNT(*) AS NumMedicado
FROM Medicamento Med, UsoMedicamento Uso, Pessoa 
WHERE Med.Codigo = Uso.CodMedicamento AND Pessoa.Id = Uso.IdPessoa 
GROUP BY Pessoa.Id)

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [44]:
SELECT NumMedicado
FROM (SELECT Pessoa.Id, COUNT(*) AS NumMedicado
FROM Medicamento Med, UsoMedicamento Uso, Pessoa 
WHERE Med.Codigo = Uso.CodMedicamento AND Pessoa.Id = Uso.IdPessoa 
GROUP BY Pessoa.Id)
WHERE NumMedicado > (SELECT AVG(NumMedicado)
FROM (SELECT Pessoa.Id, COUNT(*) AS NumMedicado
FROM Medicamento Med, UsoMedicamento Uso, Pessoa 
WHERE Med.Codigo = Uso.CodMedicamento AND Pessoa.Id = Uso.IdPessoa 
GROUP BY Pessoa.Id));

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam 
mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [55]:
--QUERY IDOSO   
SELECT AVG(NumMedicado)
    FROM (SELECT Pessoa.Id, COUNT(*) as NumMedicado FROM Pessoa, UsoMedicamento as Uso, Medicamento as Med 
    WHERE Uso.IdPessoa = Pessoa.Id AND Med.Codigo = Uso.CodMedicamento AND Pessoa.Idade > 65
    GROUP BY Pessoa.Id);

--QUERY ADULTO   
    SELECT AVG(NumMedicado)
    FROM (SELECT Pessoa.Id, COUNT(*) as NumMedicado FROM Pessoa, UsoMedicamento as Uso, Medicamento as Med 
    WHERE Uso.IdPessoa = Pessoa.Id AND Med.Codigo = Uso.CodMedicamento AND Pessoa.Idade < 65
    GROUP BY Pessoa.Id);

5

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [ ]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.